# Introduction/Business Model

The problem I will be solving with the Foursquare location database will be to help a firm confirm if the optimal location to open a hotel in Vancouver, BC is off the street from Canada Place, in Downtown Vancouver. The target audience are the shareholders of the firm trying to open the hotel. They required my services to confirm if they have selected a good place to open the hotel.

I will need to use the Foursquare Api to find main tourist destinations like stadiums, beaches, trails, hockey arenas, parks, art galleries, plazas, waterfronts, sculpture gradens, outdoor sculptures and scenic lookouts within 15km of the proposed site of the hotel.

# Data

In [8]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import requests

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



Let us get our data, below we search for venues close to Canada Place

In [9]:
radius = 15000

CLIENT_ID = 'U2GYDX22HCMBF2RGMP5BZ1ATYLX0QJU1LC3LCNU1LJYBENTD' # your Foursquare ID
CLIENT_SECRET = 'NVY5WHASZCPPPOJDSYKAWUSG3OSOLGWXS4S12WY5K1P2RNT1' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 101

latitude = 49.288635
longitude = -123.11112

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=U2GYDX22HCMBF2RGMP5BZ1ATYLX0QJU1LC3LCNU1LJYBENTD&client_secret=NVY5WHASZCPPPOJDSYKAWUSG3OSOLGWXS4S12WY5K1P2RNT1&ll=49.288635,-123.11112&v=20180604&radius=15000&limit=101'

In [10]:
results = requests.get(url).json()
'There are {} around Canada Place.'.format(len(results['response']['groups'][0]['items']))

'There are 100 around Canada Place.'

In [11]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4bbcbcf88ec3d13ab0261b28',
  'name': 'Fairmont Pacific Rim',
  'location': {'address': '1038 Canada Pl.',
   'crossStreet': 'btwn Burrard & Thurlow',
   'lat': 49.28822730314015,
   'lng': -123.11693176354468,
   'labeledLatLngs': [{'label': 'display',
     'lat': 49.28822730314015,
     'lng': -123.11693176354468}],
   'distance': 424,
   'postalCode': 'V6C 0B9',
   'cc': 'CA',
   'city': 'Vancouver',
   'state': 'BC',
   'country': 'Canada',
   'formattedAddress': ['1038 Canada Pl. (btwn Burrard & Thurlow)',
    'Vancouver BC V6C 0B9',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d1fa931735',
    'name': 'Hotel',
    'pluralName': 'Hotels',
    'shortName': 'Hotel',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 

In [12]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Fairmont Pacific Rim,Hotel,1038 Canada Pl.,btwn Burrard & Thurlow,49.288227,-123.116932,"[{'label': 'display', 'lat': 49.28822730314015...",424,V6C 0B9,CA,Vancouver,BC,Canada,"[1038 Canada Pl. (btwn Burrard & Thurlow), Van...",NaN,4bbcbcf88ec3d13ab0261b28
1,Revolver,Coffee Shop,325 Cambie St,btwn Hastings & Cordova,49.283187,-123.109288,"[{'label': 'display', 'lat': 49.2831867506594,...",620,V6B 1H7,CA,Vancouver,BC,Canada,"[325 Cambie St (btwn Hastings & Cordova), Vanc...",Gastown,4d9b41359298b1f7635b5338
2,Meat & Bread,Sandwich Place,370 Cambie St,btwn Cordova & Hastings,49.282646,-123.109499,"[{'label': 'display', 'lat': 49.2826459299748,...",677,V6B 2N3,CA,Vancouver,BC,Canada,"[370 Cambie St (btwn Cordova & Hastings), Vanc...",NaN,4cc72e8b01fb236a8d70bbba
3,Rosewood Hotel Georgia,Hotel,801 W Georgia St,at Howe St,49.283429,-123.118911,"[{'label': 'display', 'lat': 49.28342914255638...",809,V6C 1P7,CA,Vancouver,BC,Canada,"[801 W Georgia St (at Howe St), Vancouver BC V...",NaN,4d5ec8ce29ef236ae0cb9059
4,Coal Harbour Seawall,Trail,W Cordova St.,NaN,49.291304,-123.123276,"[{'label': 'display', 'lat': 49.291304, 'lng':...",931,V7J 2E1,CA,Vancouver,BC,Canada,"[W Cordova St., Vancouver BC V7J 2E1, Canada]",NaN,4ab03489f964a520706620e3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Fable,Restaurant,1944 W 4th Ave,btwn Cypress & Maple,49.267955,-123.149005,"[{'label': 'display', 'lat': 49.26795486850577...",3587,NaN,CA,Vancouver,BC,Canada,"[1944 W 4th Ave (btwn Cypress & Maple), Vancou...",NaN,4f8214cde4b0c7b856b78366
96,Burrard Liquor Store,Food & Drink Shop,2409 Burrard St,Burrard and 8th Ave,49.264544,-123.146001,"[{'label': 'display', 'lat': 49.26454439283996...",3689,V6J 3J3,CA,Vancouver,BC,Canada,"[2409 Burrard St (Burrard and 8th Ave), Vancou...",NaN,4d4ca4cd94eeb1f726f9655d
97,Via Tevere,Pizza Place,1190 Victoria Dr,at William St,49.273854,-123.065602,"[{'label': 'display', 'lat': 49.2738535806043,...",3692,V5L 4G5,CA,Vancouver,BC,Canada,"[1190 Victoria Dr (at William St), Vancouver B...",NaN,4f752184e4b0f62bd30e8d8d
98,Raisu,Japanese Restaurant,2340 West 4th Ave.,NaN,49.268244,-123.158430,"[{'label': 'display', 'lat': 49.2682441750864,...",4117,V6K 1P1,CA,Vancouver,BC,Canada,"[2340 West 4th Ave., Vancouver BC V6K 1P1, Can...",NaN,5780683c498e51523ab4851b


This is the data I will be working with.

It consists of many different columns, however, I will only be working with a couple specific features. I will use the name feature to name the venue. I will use the categories to take only some specific rows that include main tourist destinations like stadiums, beaches, trails, hockey arenas, parks, art galleries, plazas, waterfronts, sculpture gardens, outdoor sculptures and scenic lookouts. The Latitude and Longitude will be taken so I will be able to plot the locations on a map.

If there are at least 25 of the destinations specified above in the dataset, then I shall be able to come to the conclusion that the location for the hotel is optimal and we can assure that there are enough venues around the area to attract tourists and keep them busy.